# Modelo de NLP
El **Procesamiento del Lenguaje Natural** (más conocido como  NLP por su nombre en inglés, _Natural Language Processing_) es el área de estudio centrada en cómo los ordenadores entienden el lenguaje humano, lo interpretan y lo procesan. Se trata de un campo complejo en el que entran en juego diferentes disciplinas, entre las que podemos destacar la Inteligencia Artificial (AI), el _big data_ o la lingüistica.

La mayor parte de las aplicaciones creadas dentro de este campo se enfocan en la comprensión, el manejo y la generación del lenguaje natural por parte de las máquinas. Entre ellas destacan:
- Asistentes virtuales o chatbots.
- Traducción automática de textos.
- Clasificación de textos.
- Resumen de textos.
- Análisis de sentimientos.
- ... y más!

En este notebook utilizaremos el conjunto que hemos inspeccionado y adecuado para poder realizar un sencillo modelo que nos ayude a **analizar los sentimientos descritos en los diferentes tweets**. Así, el siguiente script está dividido en los siguientes bloques:
- BLOQUE A: carga de datos inspeccionados.
- BLOQUE B: preprocesamiento del texto.
- BLOQUE C: representación del texto.
- BLOQUE D: partición del conjunto de datos y balanceo.
- BLOQUE E: entrenamiento del modelo de Gradient Boosting.
- BLOQUE F: inferencia.

In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

#nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

import gensim.downloader

from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

## BLOQUE A: Carga de datos
Antes de comenzar, cargaremos los datos que han sido adecuados en nuestra fase anterior de limpieza y preprocesamiento de textos:

In [ ]:
# Carga de datos ya adecuados
df = pd.read_csv(???)

In [ ]:
# Mostramos las primeras observaciones del conjunto
???

## BLOQUE B: Preprocesamiento del texto
El preprocesamiento del texto es una fase importante dentro del Procesamiento del Lenguaje Natural (NLP). El objetivo de esta fase es la de transformar el texto en crudo, de manera que sea más fácilmente consumible por los algoritmos y modelos de Machine Learning (ML) y Deep Learning (DL) a aplicar.

Esta fase consta de diferentes pasos y no son siempre los mismos. En este caso, preprocesaremos los teewts de la siguiente manera:

1. **_Lower Casing_:** Transformar palabras de mayúsculas a minúsculas.

2. **Reemplazar URLs:** Links que comienzan por "http" o "https" o "www" son reemplazados por la palabra "URL".

3. **Reemplazar Emojis:** Reemplazar emojis usando un diccionario predefinido.

4. **Reemplazar nombres de usuario:** Reemplazar @Nombres con la palabra "USER".

5. **Eliminar _Non-Alphabets_:** Reemplazar todos los caracteres excepto dígitos and _alphabets_ por un espacio.

6. **Eliminar letras consecutivas:** 3 o más letras consecutivas son reemplazadas por 2 letras (ejemplo: "Heyyyy" por "Heyy").

7. **Eliminar palabras cortas:** Palabras con menos de 2 letras son eliminadas.

8. **Eliminar _Stopwords_:** Las _Stopwords_ son aquellas palabras en ingés que no tienen un significado específico por si solas, por lo que pueden ser ignoradas sin sacrificar el significado de la oración (ejemplos: "the", "he").

9. **_Stemming_:** Se refiere al proceso de eliminar sufijos y dar a la palabras una forma base, de modo que diferentes variaciones de una misma palabra puedan ser representadas en la misma forma (ejemplo: “walk” y “walking" son ambas reducidas a "walk").

10. **Tokenización:** Los modelos NLP normalmente analizan los textos dividiéndolos por palabras (_tokens_) y/o oraciones.

In [ ]:
# Diccionario con los distions emojis y sus significados.
emojis = {':)': 'smile', ':-)': 'smile', ';d': 'wink', ':-E': 'vampire', ':(': 'sad', 
          ':-(': 'sad', ':-<': 'sad', ':P': 'raspberry', ':O': 'surprised',
          ':-@': 'shocked', ':@': 'shocked',':-$': 'confused', ':\\': 'annoyed', 
          ':#': 'mute', ':X': 'mute', ':^)': 'smile', ':-&': 'confused', '$_$': 'greedy',
          '@@': 'eyeroll', ':-!': 'confused', ':-D': 'smile', ':-0': 'yell', 'O.o': 'confused',
          '<(-_-)>': 'robot', 'd[-_-]b': 'dj', ":'-)": 'sadsmile', ';)': 'wink', 
          ';-)': 'wink', 'O:-)': 'angel','O*-)': 'angel','(:-D': 'gossip', '=^.^=': 'cat'}

In [ ]:
# Función para preprocesar el texto en crudo
def preprocess(text):    
    # Crear stemmer.
    stemmer = SnowballStemmer(language='english')
    
    # Crear lista de stopwords
    en_stop = stopwords.words('english')
    
    # Definir patrones para reemplazar/eliminar.
    urlPattern        = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
    userPattern       = '@[^\s]+'
    alphaPattern      = "[^a-zA-Z0-9]"
    sequencePattern   = r"(.)\1\1+"
    seqReplacePattern = r"\1\1"

    # Lower Casing
    text = text.lower()
    # Reemplazar URLs
    text = re.sub(urlPattern,' URL', text)
    # Reemplazar emojis.
    for emoji in emojis.keys():
        text = text.replace(emoji, "EMOJI" + emojis[emoji])        
    # Reemplazar @Nombres con 'USER'.
    text = re.sub(userPattern,' USER', text)        
    # Reemplazar non-alphabets.
    text = re.sub(alphaPattern, " ", text)
    # Reemplazar letras consecutivas.
    text = re.sub(sequencePattern, seqReplacePattern, text)

    # Tokenizar texto
    tokens = word_tokenize(text)
    
    # Eliminar palabras con menos de dos letras
    tokens = [word for word in tokens if len(word)>2]
    
    # Eliminar stopwords
    tokens = [word for word in tokens if word not in en_stop]
    
    # Aplicar stemmer o "stemmizar"
    tokens = [stemmer.stem(word) for word in tokens]
        
    return tokens


In [ ]:
# Aplicamos la función a cada una de las filas del dataset
df['preprocess_text'] = df[???].apply(???)

In [ ]:
# Resultados del preprocesamiento: un ejemplo
print('Texto en crudo:', df.loc[16, 'text'])
print('Texto preprocesado:', df.loc[16, 'preprocess_text'])

## BLOQUE C: Representación del texto
La conversión del texto en una representación númerica es uno de los pasos más importantes dentro de cualquier _pipeline_ de NLP. Esta conversión resulta esencial para que las "máquinas" puedan comprender y decodificar patrones dentro de cualquier lenguaje.

Se trata de un proceso iterativo y que puede ser realizado mediante múltiples maneras o técnicas, abarcando desde las representaciones más sencillas (por ejemlo, _One hot encoding_) hasta otras más "inteligentes", que logran tener en cuenta las similitudes y diferencias entre ellas al basar su aprendizaje en redes neuronales (_Word embeddings_). En este [enlace](https://www.kaggle.com/code/nkitgupta/text-representations) podéis encontrar más información acerca de las diferentes técnicas normalmente empleadas.

Nosotros utilizaremos esta última técnica, sirviéndonos de un algoritmo conocido como [GloVe](https://towardsdatascience.com/light-on-math-ml-intuitive-guide-to-understanding-glove-embeddings-b13b4f19c010).

In [ ]:
# Cargamos el modelo GloVe preentrenado
GloveModel = gensim.downloader.load('glove-twitter-50')

In [ ]:
# Podemos ver la representación de una palabra
GloveModel[???]

In [ ]:
# También podemos ver las palabras con mayor similitud a otra
GloveModel.most_similar(???)

In [ ]:
# Construcción de nuestra matriz de representación

# Función para obtener/calcular el vector de representación para cada tweet
def get_w2v_vectors(processed_text, model = GloveModel):
    # Guardamos el vocabulario del modelo Word2Vec en un objeto
    words = model.index_to_key
    
    # Guardamos el tamaño de los vectores creados por el modelo en un objeto
    size = model.vector_size
    
    # Iteramos sobre los tokens del tweet para obtener su vector en el modelo
    text_vectors = []  # Lista vacía para poder guardar los vectores calculados

    for token in processed_text:        
        if token in words:
            text_vectors.append(model[token])  # Si el token existe dentro del vocabulario, añadimos el valor de su vector
            
        else:
            text_vectors.append(np.zeros(size))  # En caso de no existir, creamos un vector del mismo tamaño que sea todo 0's
    
    # Calculamos la media de todos los vectores de un tweet para poder crear una representación de todo el tweet
    text_vectors_avg = np.mean(text_vectors, axis=0)
            
    return text_vectors_avg

In [ ]:
# Aplicamos la función a todo el conjunto de datos
df['text_vector'] = df[???].apply(???)

In [ ]:
df.head()

## BLOQUE D: Partición del conjunto de datos y balanceo del conjunto de entrenamiento

In [ ]:
# Separamos nuestro dataset en dos conjuntos distintos: de entrenamiento y de test
train_set, test_set = ????(df, test_size=0.15, random_state=0)

Como habíamos visto en nuestra exploración del conjunto, sabemos que se encuentra claramente **desbalanceado** (más muestras de sentimiento negativo que de las otras dos clases). Si utilizaramos un conjunto desbalanceado para entrenar, nuestro modelo estaría claramente sesgado y le costaría aprender a diferenciar los patrones de las clases más minoritarias. Por tanto, procedemos a balancear el **conjunto de entrenamiento**:

In [ ]:
# Ver distribución de las clases en el conjunto de entrenamiento
train_set['airline_sentiment'].???

In [ ]:
# Balanceo del conjunto de entrenamiento
train_neg = train_set[???]
train_neutral = train_set[???]
train_pos = train_set[???]

num_minority = len(???)

??? = resample(???, replace=False, n_samples=num_minority, random_state=0)
??? = resample(???, replace=False, n_samples=num_minority, random_state=0)

train_set_balanced = pd.???([train_neg, train_neutral, train_pos])

In [ ]:
# Comprobamos que el conjunto esta bien balanceado
train_set_balanced['airline_sentiment'].???

In [ ]:
# Por último, separamos nuestros sets de entrenamiento y test en conjuntos X e y
X_train_balanced = list(train_set_balanced['???'])
y_train_balanced = train_set_balanced['???']

X_test = list(test_set['???'])
y_test = test_set['???']

In [ ]:
# Información acerca de los conjuntos
print('Tamaño del conjunto de entrenamiento balanceado:', len(???))
print('Tamaño del conjunto de test:', len(???))

## BLOQUE E: Entrenamiento del modelo de Gradient Boosting
¿Qué es _Boosting_?

Boosting es un meta-algoritmo de aprendizaje automático que reduce el sesgo y la varianza en un contexto de aprendizaje supervisado. Consiste en combinar los resultados de varios clasificadores débiles para obtener un clasificador robusto. Cuando se añaden estos clasificadores débiles, se hace de modo que éstos tengan diferente peso en función de la exactitud de sus predicciones. Tras añadir un clasificador débil, los datos cambian su estructura de pesos: los casos mal clasificados ganan peso y los que son clasificados correctamente pierden peso.

**Gradient Boosting (GB)** o Potenciación del gradiente consiste en plantear el problema como una optimización numérica en el que el objetivo es minimizar una función de coste añadiendo clasificadores débiles mediante el descenso del gradiente. Involucra tres elementos:

- La **función de coste** a optimizar: depende del tipo de problema a resolver.
- Un **clasificador débil** para hacer las predicciones: por lo general se usan árboles de decisión.
- Un **modelo que añade (ensambla) los clasificadores débiles** para minimizar la función de coste: se usa el descenso del gradiente para minimizar el coste al añadir árboles.

Los hiperparámetros más importantes que intervienen en este algoritmo (aunque no todos) son:
- **learning_rate**: determina el impacto de cada árbol en la salida final. Se parte de una estimación inicial que se va actualizando con la salida de cada árbol. Es el parámetro que controla la magnitud de las actualizaciones.
- **n_estimators**: número de clasificadores débiles a utilizar.

Como en este caso utilizaremos **árboles de decisión** como clasificadores débiles a ensamblar, también debemos tener en cuenta los hiperparámetros que afectan a esta clase de modelos. En este caso:
- **max_depth**: profundidad máxima del árbol.

Más información sobre el modelo que se utiliza en este ejemplo y de sus parámetros [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html).

In [ ]:
# Creamos el modelo introduciendo los valores de los parámetros:
gb_clf = ???(n_estimators=150, learning_rate=0.2, max_depth=3, random_state=0)

# Entrenamiento o ajuste del modelo con los datos de entrenamiento
gb_clf.fit(???, ???)

Para poder ver como de bueno es nuestro modelo, podemos obtener las predicciones que realiza sobre los conjuntos de entrenamiento y test, y realizar el cálculo de alguna métrica para observar su rendimiento. En este caso, observaremos las métricas **precision**, **recall** y **F1 score**, las cuales son muy utilizadas sobre todo para casos en los que se trabaja con un alto desbalanceo de los datos.

- **Precision** permite medir la calidad del modelo en tareas de clasificación. Para este ejemplo concreto, si nos centramos en los tweets postivos, mediría la cantidad de tweets positivos que nuestro modelo es capaz de identificar correctamente de entre todos los tweets que **nuestro modelo clasifica** como positivos. Para esta métrica, los falsos positivos son más importantes que los falsos negativos.

- **Recall** informa sobre la cantidad que el modelo es capaz de identificar. Siguiendo con el ejemplo anterior, esta métrica nos permite cuantificar o medir la cantidad de tweets positivos que el modelo predice correctamente de entre todos los tweets que **realmente** son positivos. Al contratio que para _precision_, para esta métrica son más importantes los falsos negativos.

- **F1 score** se utiliza para combinar ambas medidas, normalmente asumiendo que nos importan de igual forma.

Más información detallada sobre estas métricas en este [enlace](https://mlu-explain.github.io/precision-recall/).

In [ ]:
# Predecimos sobre los datos de entrenamiento
pred_train = gb_clf.predict(???)

# Mostramos el "classification report"
print('Resultados conjunto de entrenamiento:\n')
print(classification_report(y_train_balanced, pred_train))

In [ ]:
# Evaluación del modelo sobre datos de test
pred_test = gb_clf.predict(???)

# Mostramos el "classification report"
print(classification_report(y_test, pred_test))

## BLOQUE F: Inferencia 
Una vez obtenido nuestro modelo final podemos utilizarlo para realizar inferencia sobre nuevos tweets no vistos anteriormente y catalogarlos así como positivos, neutros o negativos. Debemos tener en cuenta que los tweets que vayan a ser analizados mediante este modelo NLP deben someterse al mismo preprocesamiento y representación al que se han sometido el resto de los datos.

In [ ]:
# Nuevos tweets a clasificar
new_tweets = ["Don't travel with @UnitedAirlines_ ! They lost my luggage 2 months ago on a flight to Vegas, after 5 days they asked me to fill a claim because" \
              "they didn't know where my luggage was and specify what I had spent with receipts.",
              "Virgin Atlantic and LATAM Airlines have submitted an application to the US Department of Transportation for a codeshare agreement which will create good " \
              "connectivity into three South American countries.",
              "A wonderful flight to Paris on @airfrance done and dusted. Now for a short flight to Geneva on a beaut of a plane. I love the touch of the phone/iPad holders. " \
              "@Club_Med_SA #clubmedtignes #AirFranceZA",
              "Very nice trip back from HEL with the brand new @AirFranceFR A220-330 F-HZUN from on AF1177. As usual excellent experience on board with the efficient" \
              "crew and a wonderful sunrise ! #Travel #Aircraft #a220 #airfrance #like #happy Good day everyone"]

In [ ]:
new_tweets

In [ ]:
# Preprocesamos los nuevos tweets para limpiar el texto
preprocess_new_tweets = [??? for tweet in new_tweets]

In [ ]:
# Calculamos los vectores correspondientes a cada nuevo tweet
vectors_new_tweets = [??? for tweet in preprocess_new_tweets]

In [ ]:
# Realizamos la predicción sobre estos nuevos tweets
pred_new_tweets = gb_clf.predict(???)

In [ ]:
# Vemos el resultado para cada tweet
for tweet, pred in zip(new_tweets, pred_new_tweets):
    print(tweet, '--->', pred.upper())